## Exploratory analysis 



### Load the dataset

In [2]:
import pickle as pkl
with open('datasets/outter_conj_clean.pkl', 'rb') as f:
    outter_conj_clean=pkl.load(f)
with open('datasets/outter_conj_clean.pkl', 'rb') as f:
    outter_conj_corrupt=pkl.load(f)

with open('datasets/inner_conj_clean.pkl', 'rb') as f:
    inner_conj_clean=pkl.load(f)
with open('datasets/inner_conj_clean.pkl', 'rb') as f:
    inner_conj_corrupt=pkl.load(f)


### Setting everything up for Trasnformer lens library

In [3]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio
DEBUG_MODE = False
IN_COLAB = False
print("Running as a Jupyter notebook - intended for development only!")
from IPython import get_ipython
ipython = get_ipython()
# Code to automatically update the HookedTransformer code as its edited without restarting the kernel
ipython.magic("load_ext autoreload")
ipython.magic("autoreload 2")

pio.renderers.default = "vscode"
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader
from jaxtyping import Float, Int

from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache
torch.set_grad_enabled(False)
from neel_plotly import line, imshow, scatter
import transformer_lens.patching as patching
model = HookedTransformer.from_pretrained("gpt2-small")

import plotly.io as pio
pio.renderers.default = "vscode"


def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)


def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

Running as a Jupyter notebook - intended for development only!


/tmp/ipykernel_40999/3863020717.py:9: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("load_ext autoreload")
/tmp/ipykernel_40999/3863020717.py:10: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("autoreload 2")
Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer


In [4]:
clean_logits,clean_cache=model.run_with_cache(torch.stack([inner[0] for inner in inner_conj_clean]))
corrupt_logits,corrupt_cache=model.run_with_cache(torch.stack([inner[0] for inner in inner_conj_corrupt]))
correct_answer_idx=torch.stack([out[1] for out in inner_conj_corrupt]).view(-1, 1)
incorrect_answer_idx=torch.stack([out[0][2] for out in inner_conj_corrupt]).view(-1, 1)
not_idx=model.to_tokens(" not")[0][1]
not_answer_idx=not_idx.repeat(12).reshape((12,1))

: 

: 

In [6]:
clean_logits_single,clean_cache_single=model.run_with_cache(outter_conj_clean[0][0] )

### Logit difference directions from correct answer to incorrect answer (single prompt)

In [ ]:

id=clean_logits_single[...,-1,:].argmax()
incorrect_id=outter_conj_clean[0][0][2]
logit_diff_directions = model.tokens_to_residual_directions(id)-model.tokens_to_residual_directions(incorrect_id)
logit_diff_directions=logit_diff_directions.reshape((1,768))
final_residual_stream = clean_cache_single["resid_post", -1]
scaled_final_token_residual_stream = clean_cache_single.apply_ln_to_stack(final_residual_stream, layer = -1, pos_slice=-1)



### Logit difference directions from correct answer to incorrect answer(average)

In [73]:
id_answers=clean_logits[...,-1,:].argmax()
logit_diff_directions = model.tokens_to_residual_directions(correct_answer_idx)-model.tokens_to_residual_directions(incorrect_answer_idx)
logit_diff_directions=logit_diff_directions.reshape((12,768))
final_residual_stream = clean_cache["resid_post", -1]
scaled_final_token_residual_stream = clean_cache_single.apply_ln_to_stack(final_residual_stream, layer = -1, pos_slice=-1)
scaled_final_token_residual_stream

tensor([[[-0.0420,  0.0324, -0.1105,  ...,  0.0320,  0.0604, -0.0261],
         [-0.0438,  0.0715,  0.2294,  ...,  0.0618,  0.0463, -0.1730],
         [ 0.0859,  0.4611,  0.2839,  ...,  0.1489,  0.1382, -0.5178],
         ...,
         [-0.0320,  0.1626,  0.1554,  ...,  0.4438,  0.0383, -0.3618],
         [ 0.1099,  0.2793, -0.1650,  ...,  0.3287, -0.1032, -0.4364],
         [-0.0087,  0.6320,  0.0756,  ...,  0.1045, -0.2095,  0.1253]],

        [[-0.0420,  0.0324, -0.1105,  ...,  0.0320,  0.0604, -0.0261],
         [-0.0438,  0.0715,  0.2294,  ...,  0.0618,  0.0463, -0.1730],
         [ 0.0496,  0.0668, -0.0082,  ..., -0.0975,  0.0801, -0.2820],
         ...,
         [-0.1131,  0.1269,  0.1382,  ...,  0.0921,  0.0067, -0.1710],
         [ 0.0882,  0.0709, -0.2850,  ..., -0.0051, -0.1037, -0.2276],
         [-0.1539,  0.3159, -0.0451,  ..., -0.1642, -0.7471,  0.4606]],

        [[-0.0420,  0.0324, -0.1105,  ...,  0.0320,  0.0604, -0.0261],
         [-0.0438,  0.0715,  0.2294,  ...,  0

### Logit difference directions from correct answer to " not" answer(average)

In [74]:
id_answers=clean_logits[...,-1,:].argmax()
logit_diff_directions = model.tokens_to_residual_directions(correct_answer_idx)-model.tokens_to_residual_directions(not_answer_idx)
logit_diff_directions=logit_diff_directions.reshape((12,768))
final_residual_stream = clean_cache["resid_post", -1]
scaled_final_token_residual_stream = clean_cache_single.apply_ln_to_stack(final_residual_stream, layer = -1, pos_slice=-1)
scaled_final_token_residual_stream

tensor([[[-0.0420,  0.0324, -0.1105,  ...,  0.0320,  0.0604, -0.0261],
         [-0.0438,  0.0715,  0.2294,  ...,  0.0618,  0.0463, -0.1730],
         [ 0.0859,  0.4611,  0.2839,  ...,  0.1489,  0.1382, -0.5178],
         ...,
         [-0.0320,  0.1626,  0.1554,  ...,  0.4438,  0.0383, -0.3618],
         [ 0.1099,  0.2793, -0.1650,  ...,  0.3287, -0.1032, -0.4364],
         [-0.0087,  0.6320,  0.0756,  ...,  0.1045, -0.2095,  0.1253]],

        [[-0.0420,  0.0324, -0.1105,  ...,  0.0320,  0.0604, -0.0261],
         [-0.0438,  0.0715,  0.2294,  ...,  0.0618,  0.0463, -0.1730],
         [ 0.0496,  0.0668, -0.0082,  ..., -0.0975,  0.0801, -0.2820],
         ...,
         [-0.1131,  0.1269,  0.1382,  ...,  0.0921,  0.0067, -0.1710],
         [ 0.0882,  0.0709, -0.2850,  ..., -0.0051, -0.1037, -0.2276],
         [-0.1539,  0.3159, -0.0451,  ..., -0.1642, -0.7471,  0.4606]],

        [[-0.0420,  0.0324, -0.1105,  ...,  0.0320,  0.0604, -0.0261],
         [-0.0438,  0.0715,  0.2294,  ...,  0

### Average logit difference correct and incorrect answer

In [75]:
answer_tokens=torch.stack((correct_answer_idx,correct_answer_idx)).squeeze(-1)
original_logits=clean_logits
def logits_to_ave_logit_diff(logits, answer_tokens, per_prompt=False):
    # Only the final logits are relevant for the answer
    final_logits = clean_logits[:, -1, :]
    answer_logits = final_logits.gather(dim=-1, index=answer_tokens)
    answer_logit_diff = answer_logits[:, 0] - answer_logits[:, 1]
    if per_prompt:
        return answer_logit_diff
    else:
        return answer_logit_diff.mean()
original_average_logit_diff = logits_to_ave_logit_diff(original_logits, answer_tokens)
print("Average logit difference:", logits_to_ave_logit_diff(original_logits, answer_tokens).item())


Average logit difference: -1.099471092224121


### Acumulated residual stream logit difference average

In [7]:

def residual_stack_to_logit_diff(residual_stack: Float[torch.Tensor, "components batch d_model"], cache: ActivationCache) -> float:
    scaled_residual_stack = cache.apply_ln_to_stack(residual_stack, layer = -1, pos_slice=-1)
    return einsum("n_layers batch d_model, batch d_model -> n_layers", scaled_residual_stack, logit_diff_directions)


accumulated_residual, labels = clean_cache_single.accumulated_resid(layer=-1, incl_mid=True, pos_slice=-1, return_labels=True)
logit_lens_logit_diffs = residual_stack_to_logit_diff(accumulated_residual, clean_cache_single)
line(logit_lens_logit_diffs,  hover_name=labels, title="Logit Difference From Accumulate Residual Stream")

### Acumulated residual stream logit difference average

In [72]:
def residual_stack_to_logit_diff(residual_stack: Float[torch.Tensor, "components batch d_model"], cache: ActivationCache) -> float:
    scaled_residual_stack = cache.apply_ln_to_stack(residual_stack, layer = -1, pos_slice=-1)
    return einsum("n_layers batch d_model, batch d_model -> n_layers", scaled_residual_stack, logit_diff_directions)/12


accumulated_residual, labels = clean_cache.accumulated_resid(layer=-1, incl_mid=True, pos_slice=-1, return_labels=True)
logit_lens_logit_diffs = residual_stack_to_logit_diff(accumulated_residual, clean_cache)
line(logit_lens_logit_diffs,  hover_name=labels, title="Logit Difference From Accumulate Residual Stream average")

## Layer Attribution

In [67]:
per_layer_residual, labels = clean_cache.decompose_resid(layer=-1, pos_slice=-1, return_labels=True)
per_layer_logit_diffs = residual_stack_to_logit_diff(per_layer_residual, clean_cache)
line(per_layer_logit_diffs, hover_name=labels, title="Logit Difference From Each Layer between correct and incorrect answer")

## Head Attribution

In [41]:
per_head_residual, labels = clean_cache.stack_head_results(layer=-1, pos_slice=-1, return_labels=True)
per_head_logit_diffs = residual_stack_to_logit_diff(per_head_residual, clean_cache)
per_head_logit_diffs = einops.rearrange(per_head_logit_diffs, "(layer head_index) -> layer head_index", layer=model.cfg.n_layers, head_index=model.cfg.n_heads)
imshow(per_head_logit_diffs, title="Logit Difference From Each Head")